In [1]:
import pandas as pd
import re
import numpy as np

In [2]:
df = pd.read_csv('Files/full_merge_advance_max_with_ethnicity_and_gender.csv', index_col=0)

In [8]:
import spacy
nlp = spacy.load("en_core_web_sm")

def fix_spelling_mistake_for_rows_in_full_merge_data(dx):
    dx.loc[11395,'All'] = "NYT bestselling author Mark Bego's complete and up-to-the-moment WHITNEY HOUSTON: THE BIOGRAPHY, again to Jay Cassell at Skyhorse, in a nice deal, for nearly-instant publication just in advance of Skyhorse's scheduled April publication of Bego's biography of Houston's godmother, Aretha Franklin, by Scott Mendel at the Mendel Media Group (NA).\nTranslation: scott@mendelmedia.com"
    dx.loc[2543:2544,'All'] = "Donna Alward's next three romances, again to Sally Williamson at Harlequin Romance, in a nice deal, by Jennifer Schober at Spencerhill Associates (world).\njschober@spencerhillassociates.com"
    dx.loc[13077:13085,'All'] = "Author of the Vespasian series Robert Fabbri's TO THE STRONGEST, a story of treacherous generals, scheming princesses, epic battles and exotic warrior queens, set in a historical era that echoes GAME OF THRONESÃ¢â‚¬â€\x9dthe period that followed the death of Alexander the Great, with intrigues, murders, and an eventual civil war to Sarah O'Keefe at Corvus, in a good deal, in a three-book deal, for publication in summer 2020, by Ian Drury at Sheil Land Associates (world English)."
    dx.loc[13622,'All'] = "Heidi Mastrogiovanni's LALA PETTIBONE'S ACT TWO, pitched as a humorous account of a modern-day Bridget Jones finding inspiration again after her life falls apart, to Kayla Church at Amberjack, in a nice deal, for publication in February 2017."
    dx.loc[14066,'All'] = "Matthew Wilson's THE BLACKBIRD SINGULARITY, a first-person literary novel about the struggle to start a new life after losing a child, to Lauren Parson at Legend Press, in a nice deal, by David Haviland at the Andrew Lownie Literary Agency (World English)."
    dx.loc[88:94,'All'] = "Bestselling author of historical fiction and novels featuring eponymous hero Richard Sharpe, Bernard Cornwell's next four untitled novels, announced shortly before publication of his 50th book, 1356, to Jonathan Burnham and Jennifer Barth at Harper, and Kate Elton and Katie Espiner at Harper Fiction, in a major deal, for publication through 2017, by Toby Eady at Toby Eady Associates (world English)."
    
    
def extract_from_pattern_to_name_and_name_at(dx):
    
    pattern = r"to ([\w\s]+) and ([\w\s]+) at"
    dx.loc[dx['Editor'].apply(len) > 1, 'Editor'] = np.where(dx.loc[dx['Editor'].apply(len) > 1, 'All'].str.contains(pattern),
                                                        dx.loc[dx['Editor'].apply(len) > 1, 'All'].str.extract(pattern).apply(lambda x: x.tolist(), axis=1),
                                                        np.nan)
    
def extract_from_pattern_to_name_at(dx):
    pattern = r"to ([\w\s]+) at"
    
    dx.loc[dx['Editor'].isna(), 'Editor'] = np.where(dx.loc[dx['Editor'].isna(), 'All'].str.contains(pattern),
                                                        dx.loc[dx['Editor'].isna(), 'All'].str.extract(pattern).apply(lambda x: x.tolist(), axis=1),
                                                        np.nan)
def extract_from_pattern_to_name_at_array(dx):
    pattern = r"to ([\w\s]+) at"
    
    dx.loc[dx['Editor'].apply(len)==0, 'Editor'] = np.where(dx.loc[dx['Editor'].apply(len)==0, 'All'].str.contains(pattern),
                                                        dx.loc[dx['Editor'].apply(len)==0, 'All'].str.extract(pattern).apply(lambda x: x.tolist(), axis=1),
                                                        np.nan)
def get_editor_ent_nlp_to_at(text):
    doc = nlp(text)
    names = []
    for d in doc.ents:
        '''
        if d.text in ['Vikki Ciaffone', 'Cassie Jones]':
            d.label_ = 'PERSON'
        '''
        if f"to {d.text} at" in doc.text and (d.label_ == 'PERSON' or d.label_ == 'ORG') :
            names.append(d.text)
            break
    return names

def get_editor_ent_nlp_to(text):
    doc = nlp(text)
    names = []
    for d in doc.ents:
        '''
        if d.text in ['Vikki Ciaffone', 'Cassie Jones]':
            d.label_ = 'PERSON'
           '''
        if f"to {d.text}" in doc.text and "'s" not in d.text and f"{d.text}'s" not in doc.text and (d.label_ == 'PERSON' or d.label_ == 'ORG'):#or d.label_ == 'ORG'
            names.append(d.text)
    return names

def get_editor(df, column):
    
    fix_spelling_mistake_for_rows_in_full_merge_data(df)
    df['Editor'] = df[column].apply(lambda text : get_editor_ent_nlp_to_at(text))
    #df['Editor'] = df['Editor'].astype('str')
    #df['Editor'] = df['Editor'].str.slice(start=2, stop = -2)
    
    #temp_df = df[df['Editor'] =='']
    #temp_df['Editor'] = temp_df[column].apply(lambda text : get_editor_ent_nlp_to(text))
    #temp_df['Editor'] = temp_df['Editor'].astype('str')
    #temp_df['Editor'] = temp_df['Editor'].str.slice(start=2, stop = -2)
    df.loc[df['Editor'].apply(len) == 0 , 'Editor']  = df.loc[df['Editor'].apply(len)==0, 'All'].apply(lambda text : get_editor_ent_nlp_to(text))
    #df.loc[df['Editor'].apply(lambda x: isinstance(x, list)), 'Editor'] = df.loc[df['Editor'].apply(lambda x: isinstance(x, list)), 'Editor'].apply(lambda x: ', '.join(map(str, x)))
    #df.loc[df['Editor'].apply(lambda x: isinstance(x, list)), 'Editor'] = df.loc[df['Editor'].apply(lambda x: isinstance(x, list)), 'Editor'].apply(lambda x: x[1:-1])
    extract_from_pattern_to_name_and_name_at(df)
    #fix_spelling_mistake_for_rows_in_full_merge_data(df)
    extract_from_pattern_to_name_at(df)

    df.loc[df['Editor'].isna() , 'Editor']  = df.loc[df['Editor'].isna(), 'All'].apply(lambda text : get_editor_ent_nlp_to(text))
    extract_from_pattern_to_name_at_array(df)
    #df = pd.merge(df, temp_df, on=['isbn13'])
    
    #clean if editor names has at for of
    pattern = r'\b(for|of|at)\b.*'
    df.loc[~df['Editor'].isna(),'Editor'] = df.loc[~df['Editor'].isna(),'Editor'].str.replace(pattern, '', regex=True)
    
    
    return df

def make_advance_seven_figures_from_text(df,column):
    df.loc[df['All'].str.contains('seven figures', case=False, na=False), 'Advance'] = 10**7


In [9]:
dx = get_editor(df, 'All')

<ipython-input-8-02eb6277ffdf>:16: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  dx.loc[dx['Editor'].apply(len) > 1, 'Editor'] = np.where(dx.loc[dx['Editor'].apply(len) > 1, 'All'].str.contains(pattern),
<ipython-input-8-02eb6277ffdf>:23: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  dx.loc[dx['Editor'].isna(), 'Editor'] = np.where(dx.loc[dx['Editor'].isna(), 'All'].str.contains(pattern),
<ipython-input-8-02eb6277ffdf>:29: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  dx.loc[dx['Editor'].apply(len)==0, 'Editor'] = np.where(dx.loc[dx['Editor'].apply(len)==0, 'All'].str.contains(pattern),


In [10]:
make_advance_seven_figures_from_text(dx, 'All')

In [11]:
dx.to_csv('Editor_merge_new.csv')

In [54]:
dx = pd.read_csv('Editor_merge_new.csv')

# Remove everything after 'for', 'at', 'of' including the words
pattern = r'\b(for|of|at)\b.*'

# Apply the removal of text after the keywords only if they are present
dx.loc[~dx['Editor'].isna(),'Editor'] = dx.loc[~dx['Editor'].isna(),'Editor'].str.replace(pattern, '', regex=True)


In [55]:
dx.to_csv('text.csv')

In [56]:
#['Meredith Mundy at']
dx['Editor'] = dx['Editor'].fillna(pd.Series([[]] * len(dx)))
#dx['Editor'] = dx['Editor'].apply(lambda x: x if isinstance(x, list) else [x])


In [58]:
max_elements = dx['Editor'].apply(len).max()

# Create new column names for each element index
column_names = [f'Editor{i+1}' for i in range(max_elements)]

# Split the names into different columns
dx[column_names] = dx['Editor'].apply(lambda x: pd.Series(x[:max_elements]))


ValueError: Columns must be same length as key

In [120]:
dx[dx['Editor'].apply(len)==0]['All'][167] #14067 #14042

"Author of MY SISTER'S SECRET Tracy Buchanan's THE WEIGHT OF SILENCE, in which a physiotherapist comes home from work to find her husband unconscious on the floor with a stab wound and their three children standing around him; it's clear he has been stabbed by one of them, but none of them is willing to say who did it or why, and THE WOMAN IN THE WOODS, to Sammia Hamer at Lake Union Publishing, in a very nice deal, by Caroline Hardman at Hardman & Swainson (world English).\ncaroline@hardmanswainson.com\nRights: Therese Coen at Hardman & Swainson"

In [95]:
dx[dx['Editor'].apply(len)>0]['All'][88]#14067 #14042

"Bestselling author of historical fiction and novels featuring eponymous hero Richard Sharpe, Bernard Cornwell's next four untitled novels, announced shortly before publication of his 50th book, 1356, to Jonathan Burnham andJennifer Barth at Harper, and Kate Elton and Katie Espiner at Harper Fiction, in a major deal, for publication through 2017, by Toby Eady at Toby Eady Associates (world English)."

In [121]:
td= nlp("Author of MY SISTER'S SECRET Tracy Buchanan's THE WEIGHT OF SILENCE, in which a physiotherapist comes home from work to find her husband unconscious on the floor with a stab wound and their three children standing around him; it's clear he has been stabbed by one of them, but none of them is willing to say who did it or why, and THE WOMAN IN THE WOODS, to Sammia Hamer at Lake Union Publishing, in a very nice deal, by Caroline Hardman at Hardman & Swainson (world English).\ncaroline@hardmanswainson.com\nRights: Therese Coen at Hardman & Swainson")

In [122]:
for d in td.ents:
    print(d, d.label_)

Tracy Buchanan's PERSON
three CARDINAL
one CARDINAL
Sammia Hamer GPE
Lake Union Publishing ORG
Caroline Hardman PERSON
Hardman & Swainson ORG
English LANGUAGE
Hardman & Swainson ORG


In [56]:
import nltk
from nltk import ne_chunk
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/rathod.rak/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /home/rathod.rak/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /home/rathod.rak/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


True

In [57]:
sentences = nltk.sent_tokenize("Matthew Wilson's THE BLACKBIRD SINGULARITY, a first-person literary novel about the struggle to start a new life after losing a child, to Lauren Parsons at Legend Press, in a nice deal, by David Haviland at the Andrew Lownie Literary Agency (World English).")
words = [nltk.word_tokenize(sentence) for sentence in sentences]

In [58]:
tagged_words = [nltk.pos_tag(sentence) for sentence in words]

In [59]:
named_entities = [ne_chunk(tagged_sentence) for tagged_sentence in tagged_words]

In [61]:
for named_entity in named_entities:
    for subtree in named_entity.subtrees():
        if subtree.label() in ['PERSON', 'ORGANIZATION', 'LOCATION']:
            entity = ' '.join([word for word, tag in subtree.leaves()])
            entity_type = subtree.label()
            print(entity, entity_type)

Matthew PERSON
Wilson PERSON
THE ORGANIZATION
BLACKBIRD ORGANIZATION
Lauren Parsons PERSON
Legend Press ORGANIZATION
David Haviland PERSON
Andrew Lownie Literary Agency PERSON


In [21]:
pattern = r"to ([\w\s]+) and ([\w\s]+) at"

In [22]:
matches[['Editor', 'Editor2']] = dx[dx['Editor'].apply(len)>1]['All'].str.extract(pattern)

In [35]:
dx.loc[dx['Editor'].apply(len) > 1, 'Editor'] = np.where(dx.loc[dx['Editor'].apply(len) > 1, 'All'].str.contains(pattern),
                                                        dx.loc[dx['Editor'].apply(len) > 1, 'All'].str.extract(pattern).apply(lambda x: x.tolist(), axis=1),
                                                        np.nan)

<ipython-input-35-660f68b67564>:1: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  dx.loc[dx['Editor'].apply(len) > 1, 'Editor'] = np.where(dx.loc[dx['Editor'].apply(len) > 1, 'All'].str.contains(pattern),


In [57]:
pattern = r"to ([\w\s]+) at"

In [58]:
dx.loc[dx['Editor'].isna(), 'Editor'] = np.where(dx.loc[dx['Editor'].isna(), 'All'].str.contains(pattern),
                                                        dx.loc[dx['Editor'].isna(), 'All'].str.extract(pattern).apply(lambda x: x.tolist(), axis=1),
                                                        np.nan)

<ipython-input-58-ce4045569917>:1: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  dx.loc[dx['Editor'].isna(), 'Editor'] = np.where(dx.loc[dx['Editor'].isna(), 'All'].str.contains(pattern),


In [71]:
dx.loc[dx['Editor'].isna() , 'Editor']  = dx.loc[dx['Editor'].isna(), 'All'].apply(lambda text : get_editor_ent_nlp_to(text))


In [84]:
dx[dx['Editor'].apply(len)>2]['All'][5756]

"Rights to Tui Sutherland's NY Times Bestselling WINGS OF FIRE series, to Gallimard in France, in a significant deal, in a 6-publisher auction, by Noemie Rollet at Eliane Benisti Agency; to Loewe in Germany, by Friederike Biesel at the Thomas Schlueck Agency; to Fundamento in Brazil, by Joao Paulo Riff at the Riff Agency; to Aspendos in Turkey, by Bengu Ayfer at Akcali Copyright Agency, all on behalf of Cecilia de la Campa and Steven Malk at Writers House."

In [69]:
dx[dx['Editor'].isna()]['All'][12909]#[5089]

"Drew Hayes's UNDEATH AND TAXES, the sequel to THE UTTERLY UNINTERESTING AND UNADVENTUROUS TALES OF FRED, THE VAMPIRE ACCOUNTANT, joining Fred on a new series of mini-adventures where not only the living dread tax season, to Reuts, in a nice deal, for publication in Summer 2015 (World English)."

In [46]:
type(dx['Editor'][572])

list

In [56]:
dx[dx['Editor'].isna()]['All'][662]

"Lisa Greenwald's THE SUMMER OF PINK & GREEN, sequel to MY LIFE IN PINK & GREEN, in which the family's new eco-spa business spawns as many mishaps as it does makeovers, to Maggie Lehrman at Amulet, in a very nice deal, for publication in 2012, by Alyssa Eisner Henkin at Trident Media Group (World English)."

In [40]:
dx[dx['Editor'].apply(len)>1]['Editor']

TypeError: object of type 'float' has no len()

In [116]:
dc = nlp(text)

In [117]:
dc.ents

(Kristen Ciccarelli's,
 Kristen Pettit at Harper Teen,
 three,
 Fall 2017,
 Heather Flaherty,
 The Bent Agency,
 NA,
 Gollancz,
 UK,
 Gemma Cooper,
 The Bent Agency,
 Heather Flaherty,
 Gallimard Jeunesse,
 France,
 Corinne Marotte,
 L'Autre Agence,
 Heyne,
 Germany,
 Bastian Schlueck,
 Thomas Schlueck Agency,
 Nocturna,
 Spain,
 Philip Sane,
 Lennart Sane Agency,
 Companhia das Letras,
 Brazil,
 Philip Sane,
 Lennart Sane Agency,
 Poland,
 Magda Cabajewska,
 the Macadamia Literary Agency,
 Dogan Egmont,
 Turkey,
 Asli Ermis,
 Asli Karasuil Telif Literary Agency,
 Blossom Books,
 Netherlands,
 Lester Hekking of Sebes & Bisseling,
 Dana Spector)

In [83]:
dx.loc[dx['Editor'].apply(lambda x: isinstance(x, list)), 'Editor'] = dx.loc[dx['Editor'].apply(lambda x: isinstance(x, list)), 'Editor'].apply(lambda x: ', '.join(map(str, x)))
dx.loc[dx['Editor'].apply(lambda x: isinstance(x, list)), 'Editor'] = dx.loc[dx['Editor'].apply(lambda x: isinstance(x, list)), 'Editor'].apply(lambda x: x[1:-1])


In [86]:
dx.to_csv('test.csv')

In [27]:
#dx = pd.read_csv('Files/Editor_merge.csv')

In [44]:
text = dx['All'][1]

In [66]:
doc = nlp(text)

In [68]:
for d in doc.ents:
    if "'s" in d.text:
        print(d)

Hilary Mantel's
